In [1]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset
import numpy
import sqlite3
import pandas as pd
import numpy as np
from sklearn import model_selection
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from bs4 import BeautifulSoup
# fix random seed for reproducibility
numpy.random.seed(7)

def savefile(obj,filename):
    pickle.dump(obj,open(filename+".p","wb"))
def openfile(filename):
    temp = pickle.load(open(filename+".p","rb"))
    return temp
from tqdm import tqdm
import os


Using TensorFlow backend.


In [2]:
if os.path.isfile('final.sqlite'):
    conn = sqlite3.connect('final.sqlite')
    final = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, conn)
    conn.close()
else:
    print("Please the above cell")
    

In [27]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [29]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [6]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 364171/364171 [01:49<00:00, 3320.65it/s]


In [7]:
final['CleanedText'].values[0]

'witti littl book make son laugh loud recit car drive along alway sing refrain hes learn whale india droop love new word book introduc silli classic book will bet son still abl recit memori colleg'

In [228]:
preprocessed_reviews[0]

'witty little book makes son laugh loud recite car driving along always sing refrain learned whales india drooping roses love new words book introduces silliness classic book willing bet son still able recite memory college'

In [8]:
final['preprocessed_reviews']=preprocessed_reviews

In [3]:
final.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak book watch realli rosi movi i...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn month year learn poem t...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great littl book read nice rhythm well good re...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetri month year goe month cute littl po...


In [4]:


final["Time"] = pd.to_datetime(final["Time"] , unit = "s")
reviews_timesorted = final.sort_values(by = "Time")
print (reviews_timesorted.shape)
reviews_timesorted_score = reviews_timesorted["Score"]
print (reviews_timesorted_score.shape)
reviews_timesorted.head(5)

(364171, 12)
(364171,)


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,1999-10-08,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
30,138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,1,1999-10-25,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,rememb see show air televis year ago child sis...
424,417839,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,1,1999-12-02,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,beetlejuic well written movi everyth excel act...
330,346055,374359,B00004CI84,A344SMIA5JECGM,Vincent P. Ross,1,2,1,1999-12-06,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",twist rumplestiskin captur film star michael k...
423,417838,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,1,2000-01-03,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,beetlejuic excel funni movi keaton hilari wack...


In [5]:

# reviews_100k = reviews_timesorted.head(100000)
reviews_100k = reviews_timesorted.head(100000)

print (reviews_100k.shape)


reviews_100k_score = reviews_100k["Score"]
print (reviews_100k_score.shape)

reviews_100k.head()



(100000, 12)
(100000,)


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,1999-10-08,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
30,138683,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,1,1999-10-25,This whole series is great way to spend time w...,I can remember seeing the show when it aired o...,rememb see show air televis year ago child sis...
424,417839,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,1,1999-12-02,Entertainingl Funny!,Beetlejuice is a well written movie ..... ever...,beetlejuic well written movi everyth excel act...
330,346055,374359,B00004CI84,A344SMIA5JECGM,Vincent P. Ross,1,2,1,1999-12-06,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta...",twist rumplestiskin captur film star michael k...
423,417838,451855,B00004CXX9,AJH6LUC1UT1ON,The Phantom of the Opera,0,0,1,2000-01-03,FANTASTIC!,Beetlejuice is an excellent and funny movie. K...,beetlejuic excel funni movi keaton hilari wack...


### Converting words to ids and taking top 5000 words based on the occurrence :

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer()
reviews_vocab = count_vec.fit_transform(reviews_100k['CleanedText'].values)


In [7]:
#Reference : https://stackoverflow.com/questions/32194508/convert-a-list-of-words-to-a-list-of-integers-in-scikit-learn

word_count =  list(zip(count_vec.get_feature_names(),\
    np.asarray(reviews_vocab.sum(axis=0)).ravel()))

word_count_sorted = sorted(word_count , key=lambda x:x[1] , reverse=True)

In [8]:
print ("Taking top 5000 words from the vocab")
word_count_sorted = word_count_sorted[:5000]

Taking top 5000 words from the vocab


In [9]:
word_count_sorted = { j[0]:i  for i,j in enumerate(word_count_sorted,1) }

In [10]:
len(word_count_sorted)

5000

In [11]:
word_count_sorted.items()

dict_items([('like', 1), ('tast', 2), ('good', 3), ('flavor', 4), ('tea', 5), ('great', 6), ('product', 7), ('use', 8), ('one', 9), ('love', 10), ('tri', 11), ('make', 12), ('get', 13), ('coffe', 14), ('would', 15), ('food', 16), ('buy', 17), ('time', 18), ('eat', 19), ('amazon', 20), ('realli', 21), ('best', 22), ('find', 23), ('also', 24), ('dont', 25), ('order', 26), ('much', 27), ('price', 28), ('littl', 29), ('even', 30), ('drink', 31), ('ive', 32), ('store', 33), ('bag', 34), ('well', 35), ('mix', 36), ('better', 37), ('chocol', 38), ('sugar', 39), ('box', 40), ('year', 41), ('dog', 42), ('recommend', 43), ('day', 44), ('sweet', 45), ('found', 46), ('high', 47), ('first', 48), ('want', 49), ('look', 50), ('cup', 51), ('treat', 52), ('give', 53), ('enjoy', 54), ('brand', 55), ('water', 56), ('way', 57), ('think', 58), ('packag', 59), ('made', 60), ('delici', 61), ('add', 62), ('purchas', 63), ('favorit', 64), ('work', 65), ('bought', 66), ('nice', 67), ('two', 68), ('snack', 69), 

In [12]:
word_count_sorted.get('good',0) #Get the id . If not id than default 0

3

In [13]:
# Refer sklearn count-veectorizer build_analyzer to convert sentence into tokens .
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.build_analyzer

tokenizer = count_vec.build_analyzer()

In [14]:
# Converting each sentences into tokens and passing tokens to get ids of the words .

sentence_ids = []
for line in tqdm(reviews_100k['CleanedText'].values) :
    sentence = []
    for x in tokenizer(line):
        sentence.append( word_count_sorted.get(x,0) )
        
    sentence_ids.append(sentence)
        

100%|██████████| 100000/100000 [00:02<00:00, 46945.45it/s]


In [15]:
sentence_ids = np.array(sentence_ids)
sentence_ids.shape

(100000,)

In [16]:
sentence_ids[0]

[0,
 29,
 897,
 12,
 327,
 2377,
 3007,
 0,
 1075,
 1208,
 534,
 106,
 3278,
 0,
 868,
 808,
 0,
 1458,
 0,
 10,
 195,
 799,
 897,
 968,
 2886,
 1109,
 897,
 1578,
 1813,
 327,
 91,
 256,
 0,
 1261,
 1620]

### Splitting data :

In [17]:
from sklearn import model_selection

reviews_train, reviews_test, reviews_score_train , reviews_score_test = model_selection.train_test_split(sentence_ids, reviews_100k_score, test_size=0.3, random_state=0)
print (len(reviews_train))
print (len(reviews_score_train))
print (len(reviews_test))
print (len(reviews_score_test))

70000
70000
30000
30000


In [18]:
max_review_length = 500
X_train = sequence.pad_sequences(reviews_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(reviews_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(70000, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [19]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(5000+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160032    
_________________________________________________________________
lstm_1 (LSTM)                (None, 500, 100)          53200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 293,733
Trainable params: 293,733
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.fit(X_train, reviews_score_train, nb_epoch=10, batch_size=64,validation_data=(X_test, reviews_score_test))
# Final evaluation of the model
scores = model.evaluate(X_test, reviews_score_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/anaconda3/envs/yolo/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 1001s 14ms/step - loss: 0.2269 - acc: 0.9146 - val_loss: 0.1981 - val_acc: 0.9235
Epoch 2/10
70000/70000 [==============================] - 990s 14ms/step - loss: 0.1691 - acc: 0.9361 - val_loss: 0.1900 - val_acc: 0.9273
Epoch 3/10
70000/70000 [==============================] - 970s 14ms/step - loss: 0.1546 - acc: 0.9414 - val_loss: 0.1895 - val_acc: 0.9267
Epoch 4/10
70000/70000 [==============================] - 991s 14ms/step - loss: 0.1422 - acc: 0.9467 - val_loss: 0.1997 - val_acc: 0.9285
Epoch 5/10
70000/70000 [==============================] - 944s 13ms/step - loss: 0.1273 - acc: 0.9522 - val_loss: 0.2012 - val_acc: 0.9250
Epoch 6/10
70000/70000 [==============================] - 1322s 19ms/step - loss: 0.1156 - acc: 0.9578 - val_loss: 0.2055 - val_acc: 0.9245
Epoch 7/10
70000/70000 [==============================] - 976s 14ms/step - loss: 0.1012 - acc: 0.9638 - val_loss: 

In [21]:
model.save_weights('amazon_fine_food_epoch_10')

### Testing the trained model on food review :

In [26]:
def clean_text(sentence):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = BeautifulSoup(sentence, 'lxml').get_text()
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    # https://gist.github.com/sebleier/554b280
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    return sentence.strip()

In [49]:
reviews_score_train[reviews_score_train==1].shape

(215022,)

In [79]:
test_text = ["Pizza was good"]

In [80]:
review_text = clean_text(test_text[0])

In [81]:
test_text = [review_text]

In [82]:
sentence_test_ids = []
for line in tqdm(test_text) :
    print (tokenizer(line))
    sentence_test = []
    for x in tokenizer(line):
        sentence_test.append( word_count_sorted.get(x,0) )
        
    sentence_test_ids.append(sentence_test)

100%|██████████| 1/1 [00:00<00:00, 1851.79it/s]

['pizza', 'good']


In [83]:
X_test_sent = sequence.pad_sequences(sentence_test_ids, maxlen=max_review_length)

In [84]:
print (model.predict(X_test_sent)[0][0])
if model.predict(X_test_sent)[0][0] <0.50 :
    print ("negative")
else :
    print ("positive")


0.99879897
positive


## Conclusion :

- To preserve the sequence of reviews for LSTMs , words are converted to ids and it is feed into the LSTM network.
- Word ids are decided based on frequencies in the corpous . After cleaning the reviews , we use count vectorizer (BoW) to count the occurrences(frequency) of word and take top 5000 words . 
- Every sentence is padded or truncated if the length is less or more than 500 . The benefit of making reviews of same size is when training with batches in LSTM it increases the speed of training . 
- In LSTM , the sequence of reviews is maintained same as the context of word is preserved in word2vec . 